# Stage 2 Inference Demo

Demonstrate the complete two-stage pipeline:
- Stage 1: Signal detection
- Stage 2: Signal type classification
- Combined: Trading action (BUY/SELL/NONE)

In [ ]:
import sys
from pathlib import Path
import pickle
import numpy as np
import pandas as pd

project_root = Path('.').resolve().parent
sys.path.insert(0, str(project_root))

from src.stage2_inference import PipelineInference

### Step 1: Load Test Data

In [ ]:
data_dir = Path('data/stage2')

with open(data_dir / 'X_stage2_test.pkl', 'rb') as f:
    X_stage2_test = pickle.load(f)
with open(data_dir / 'y_stage2_test.pkl', 'rb') as f:
    y_stage2_test = pickle.load(f)

print(f'Test data loaded: {X_stage2_test.shape}')

### Step 2: Initialize Pipeline

In [ ]:
pipeline = PipelineInference(model_dir='models')

# Load both stages
pipeline.load_stage1_model('btcusdt_15m')
pipeline.load_stage2_model('btcusdt_15m')

### Step 3: Run Inference on Test Data

In [ ]:
predictions = pipeline.predict(X_stage2_test, return_confidence=True)

### Step 4: Display Sample Predictions

In [ ]:
# Show first 20 predictions
print('\nSample Predictions (First 20):')
print('-' * 80)
print(f'{"Index":<6} {"Stage1":<8} {"Stage2":<8} {"Action":<8} {"S1 Conf":<10} {"S2 Conf":<10} {"Combined":<10}')
print('-' * 80)

for i in range(min(20, len(X_stage2_test))):
    s1_conf = predictions['confidence']['stage1'][i]
    s2_conf = predictions['confidence']['stage2'][i]
    combined_conf = predictions['confidence']['combined'][i]
    
    print(f'{i:<6} {predictions["stage1_signal"][i]:<8} {predictions["stage2_type_name"][i]:<8} {predictions["action"][i]:<8} {s1_conf:<10.4f} {s2_conf:<10.4f} {combined_conf:<10.4f}')

print('-' * 80)

### Step 5: Get Summary Statistics

In [ ]:
stats = pipeline.get_summary_stats(predictions)
pipeline.print_summary(stats)

### Step 6: Signal Type Distribution

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Signal Distribution
signal_counts = [stats['stage1_signal'], stats['stage1_no_signal']]
axes[0].bar(['Signal', 'No Signal'], signal_counts, color=['green', 'red'])
axes[0].set_title('Stage 1: Signal Detection')
axes[0].set_ylabel('Count')

# Plot 2: Action Distribution
action_counts = [stats['buy_signals'], stats['sell_signals'], stats['no_signals']]
axes[1].bar(['BUY', 'SELL', 'NONE'], action_counts, color=['green', 'red', 'gray'])
axes[1].set_title('Final Trading Actions')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

### Step 7: Confidence Distribution

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(predictions['confidence']['stage1'], bins=30, color='blue', alpha=0.7)
axes[0].set_title('Stage 1 Confidence Distribution')
axes[0].set_xlabel('Confidence')
axes[0].set_ylabel('Frequency')

axes[1].hist(predictions['confidence']['stage2'], bins=30, color='green', alpha=0.7)
axes[1].set_title('Stage 2 Confidence Distribution')
axes[1].set_xlabel('Confidence')
axes[1].set_ylabel('Frequency')

axes[2].hist(predictions['confidence']['combined'], bins=30, color='purple', alpha=0.7)
axes[2].set_title('Combined Confidence Distribution')
axes[2].set_xlabel('Confidence')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### Step 8: Single Sample Prediction

In [ ]:
# Predict for a single sample
sample_idx = 0
X_single = X_stage2_test[sample_idx:sample_idx+1]

result = pipeline.predict_single(X_single, return_confidence=True)

print(f'\nSample {sample_idx} Prediction:')
print(f'  Has Signal: {result["has_signal"]}')
print(f'  Signal Type: {result["signal_type"]}')
print(f'  Action: {result["action"]}')
print(f'  Stage 1 Confidence: {result["stage1_confidence"]:.4f}')
print(f'  Stage 2 Confidence: {result["stage2_confidence"]:.4f}')
print(f'  Combined Confidence: {result["combined_confidence"]:.4f}')

### Step 9: Conclusion

In [ ]:
print(f'\n' + '='*60)
print('TWO-STAGE PIPELINE INFERENCE COMPLETE')
print('='*60)
print(f'\nPipeline successfully combined:')
print(f'  - Stage 1: Signal Detection (Binary Classification)')
print(f'  - Stage 2: Signal Type Prediction (4-class Classification)')
print(f'\nOverall Performance:')
print(f'  - Average Stage 1 Confidence: {stats["average_stage1_confidence"]:.4f}')
print(f'  - Average Stage 2 Confidence: {stats["average_stage2_confidence"]:.4f}')
print(f'  - Average Combined Confidence: {stats["average_combined_confidence"]:.4f}')
print(f'\nReady for production deployment!')
print('='*60)